<a href="https://colab.research.google.com/github/pachterlab/GRNP_2020/blob/master/notebooks/R_processing/ProcessR_PBMC_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PBMC_V3 dataset: Processes the BUG files into files prepared for use in R**

This notebook processes the output from the fastq file processing for this dataset. The data produced here is pre-generated and downloaded by the figure generation code. The purpose of this processing step is to prepare the data for figure generation, by filtering the data and producing downsampled datasets in addition to the original one. 

Steps:
1. Clone the code repo and download data to process
2. Prepare the R environment
3. Process the data
4. Generate statistics for the dataset

**1. Clone the code repo and download data to process**

In [1]:
![ -d "GRNP_2020" ] && rm -r GRNP_2020

!git clone https://github.com/pachterlab/GRNP_2020.git


Cloning into 'GRNP_2020'...
remote: Enumerating objects: 258, done.
remote: Counting objects: 100% (258/258), done.
remote: Compressing objects: 100% (211/211), done.
remote: Total 1141 (delta 176), reused 83 (delta 47), pack-reused 883
Receiving objects: 100% (1141/1141), 7.53 MiB | 18.99 MiB/s, done.
Resolving deltas: 100% (730/730), done.


In [2]:
#download BUG data from Zenodo
!mkdir data
!cd data && wget https://zenodo.org/record/3924675/files/PBMC_V3.zip?download=1 && unzip 'PBMC_V3.zip?download=1' && rm 'PBMC_V3.zip?download=1'

--2020-07-02 22:10:11--  https://zenodo.org/record/3924675/files/PBMC_V3.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 359563806 (343M) [application/octet-stream]
Saving to: ‘PBMC_V3.zip?download=1’

PBMC_V3.zip?downloa 100%[===================>] 342.91M  26.1MB/s    in 14s     

2020-07-02 22:10:26 (23.9 MB/s) - ‘PBMC_V3.zip?download=1’ saved [359563806/359563806]

Archive:  PBMC_V3.zip?download=1
   creating: PBMC_V3/
   creating: PBMC_V3/bus_output/
  inflating: PBMC_V3/bus_output/bug.txt  
  inflating: PBMC_V3/bus_output/coll.genes.txt  
  inflating: PBMC_V3/bus_output/transcripts_to_genes.txt  


In [3]:
#Check that download worked
!cd data && ls -l && cd PBMC_V3/bus_output && ls -l

total 4
drwxr-xr-x 3 root root 4096 Jul  1 00:00 PBMC_V3
total 1714384
-rw-r--r-- 1 root root 1746443873 Jun 30 20:47 bug.txt
-rw-r--r-- 1 root root     738211 Jun 30 19:43 coll.genes.txt
-rw-r--r-- 1 root root    8335642 Jun 30 19:39 transcripts_to_genes.txt


**2. Prepare the R environment**

In [4]:
#switch to R mode
%reload_ext rpy2.ipython


/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [5]:
#install the R packages
%%R
install.packages("qdapTools")
install.packages("dplyr")
install.packages("stringdist")
install.packages("stringr")


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘bitops’, ‘chron’, ‘data.table’, ‘RCurl’, ‘XML’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/bitops_1.0-6.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 8734 bytes

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[writ

**3. Process the data**

Here we discard multimapped UMIs and all UMIs belonging to cells with fewer than 200 UMIs. We also precalculate gene expression, fraction of single-copy molecules etc. and save as stats (statistics). These can later be used when generating figures. We also generate down-sampled BUGs.

In [6]:
#create output directory
!mkdir figureData

In [7]:
#First set some path variables
%%R
source("GRNP_2020/RCode/pathsGoogleColab.R")


In [8]:
#Process and filter the BUG file
%%R
source(paste0(sourcePath, "BUGProcessingHelpers.R"))
createStandardBugsData(paste0(dataPath,"PBMC_V3/"), "PBMC_V3", c(0.05, 0.1, 0.2, 0.4, 0.6, 0.8, 1))



R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


R[write to console]: 
Attaching package: ‘qdapTools’


R[write to console]: The following object is masked from ‘package:dplyr’:

    id




[1] "Generating data for PBMC_V3"
[1] "Reading BUG from data/PBMC_V3/ ..."
[1] "Filtering multi-mapped reads..."
[1] "Fraction multi-mapped reads: 0.178660156842713"
[1] "Converting genes..."
[1] "Done"
[1] "Down-sampling in total 7 bugs:"
[1] "7: Down-sampling to 1"
[1] "saving BUG..."
[1] "creating stats..."


R[write to console]: `summarise()` ungrouping output (override with `.groups` argument)



[1] "cpm normalizing..."
[1] "6: Down-sampling to 0.8"
[1] "1"
[1] "2"
[1] "3"
[1] "4"
[1] "5"
[1] "6"
[1] "7"
[1] "8"
[1] "9"
[1] "saving BUG..."
[1] "creating stats..."


R[write to console]: `summarise()` ungrouping output (override with `.groups` argument)



...

[1] "cpm normalizing..."
[1] "2: Down-sampling to 0.1"
[1] "1"
[1] "2"
[1] "3"
[1] "4"
[1] "5"
[1] "6"
[1] "7"
[1] "8"
[1] "9"
[1] "saving BUG..."
[1] "creating stats..."


R[write to console]: `summarise()` ungrouping output (override with `.groups` argument)



[1] "cpm normalizing..."
[1] "1: Down-sampling to 0.05"
[1] "1"
[1] "2"
[1] "3"
[1] "4"
[1] "5"
[1] "6"
[1] "7"
[1] "8"
[1] "9"
[1] "saving BUG..."
[1] "creating stats..."


R[write to console]: `summarise()` ungrouping output (override with `.groups` argument)



[1] "cpm normalizing..."
[1] "Done"
[1] "Saving stats..."


**4. Generate statistics for the dataset**

Here we create a file with various statistics for the dataset, which is used for generating table S2. It also contains some additional information about the dataset. Generation of this file may take several hours.

In [ ]:
%%R
source(paste0(sourcePath, "GenBugSummary.R"))
genBugSummary("PBMC_V3", "FGF23", "RPS10", 10)

[1] "Down-sampling to 2000 UMIs"
[1] "Will process 2000 UMIs"
[1] 100
[1] 200
[1] 300
[1] 400
[1] 500
[1] 600
[1] 700
[1] 800
[1] 900
[1] 1000
[1] 1100
[1] 1200
[1] 1300
[1] 1400
[1] 1500
[1] 1600
[1] 1700
[1] 1800
[1] 1900
[1] 2000
[1] "Down-sampling to 2000 UMIs"
[1] "Will process 2000 UMIs"
[1] 100
[1] 200
[1] 300
[1] 400
[1] 500
[1] 600
[1] 700
[1] 800
[1] 900
[1] 1000
[1] 1100
[1] 1200
[1] 1300
[1] 1400
[1] 1500
[1] 1600
[1] 1700
[1] 1800
[1] 1900
[1] 2000
[1] "Down-sampling to 2000 UMIs"
[1] "Will process 2000 UMIs"
[1] 100
[1] 200
[1] 300
[1] 400
[1] 500
[1] 600
[1] 700
[1] 800
[1] 900
[1] 1000
[1] 1100
[1] 1200
[1] 1300
[1] 1400
[1] 1500
[1] 1600
[1] 1700
[1] 1800
[1] 1900
[1] 2000
[1] "Down-sampling to 2000 UMIs"
[1] "Will process 2000 UMIs"
[1] 100
[1] 200
[1] 300
[1] 400
[1] 500
[1] 600
[1] 700
[1] 800
[1] 900
[1] 1000
[1] 1100
[1] 1200
[1] 1300
[1] 1400
[1] 1500
[1] 1600
[1] 1700
[1] 1800
[1] 1900
[1] 2000
[1] "Down-sampling to 2000 UMIs"
[1] "Will process 2000 UMIs"
[1] 10

In [ ]:
!cd figureData/PBMC_V3 && ls -l && cat ds_summary.txt

total 1152060
-rw-r--r-- 1 root root 298563353 Jul  1 23:44 Bug_100.RData
-rw-r--r-- 1 root root  59652076 Jul  1 23:32 Bug_10.RData
-rw-r--r-- 1 root root 107487769 Jul  1 23:33 Bug_20.RData
-rw-r--r-- 1 root root 179478368 Jul  1 23:35 Bug_40.RData
-rw-r--r-- 1 root root  31642931 Jul  1 23:31 Bug_5.RData
-rw-r--r-- 1 root root 230346911 Jul  1 23:38 Bug_60.RData
-rw-r--r-- 1 root root 270692069 Jul  1 23:42 Bug_80.RData
-rw-r--r-- 1 root root       986 Jul  2 02:30 ds_summary.txt
-rw-r--r-- 1 root root   1824632 Jul  1 23:44 Stats.RData
Dataset: PBMC_V3

totUMIs: 30427843
totCells: 5428
totCounts: 64154578
countsPerUMI: 2.10841688646809
UMIsPerCell: 5605.71904937362
countsPerCell: 11819.1927044952
totFracOnes: 0.453309260206187
FracMolWithUMIDistToNeighborH: 61, 588, 1181, 166, 4, 0, 0, 0, 0, 0
FracMolWithUMIDistToNeighborL: 64, 592, 1137, 197, 10, 0, 0, 0, 0, 0
FracMolWithUMIDistToNeighborHFrac: 0.0305, 0.294, 0.5905, 0.083, 0.002, 0, 0, 0, 0, 0
FracMolWithUMIDistToNeighborLFrac: 0